In [1]:
import pandas as pd

# Load your Excel file
df = pd.read_excel("test01.xlsx")

# Combine columns into a single document string for each row
df["content"] = (
    df["Systeme"].astype(str) + " | " +
    df["Description"].astype(str) + " | " +
    df["Description de l'équipement"].astype(str)
)


In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

df["embedding"] = model.encode(df["content"].tolist()).tolist()



ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
import faiss

# Convert to numpy array
embedding_matrix = np.array(df["embedding"].tolist()).astype("float32")

# Build FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)


In [ ]:
def ask_rag(query, top_k=5):
    query_embedding = np.array(get_embedding(query)).astype("float32").reshape(1, -1)
    D, I = index.search(query_embedding, top_k)
    contexts = df.iloc[I[0]]["content"].tolist()

    # Concatenate retrieved context
    prompt = "Voici des anomalies liées aux équipements:\n"
    for i, ctx in enumerate(contexts):
        prompt += f"{i+1}. {ctx}\n"
    prompt += f"\nÀ partir de ces informations, réponds à la question suivante:\n{query}"

    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Example
print(ask_rag("Quels sont les anomalies qui peuvent causer une perte de sécurité de process ?"))
